In [1]:
import pandas as pd

In [114]:
data = pd.concat([pd.read_csv('rurur.csv'), pd.read_csv('final.csv').drop(columns=['Unnamed: 0'])], axis=0, sort=True)

In [115]:
data = data.drop_duplicates()

In [116]:
data.theme.value_counts()

science     35909
politics    31149
world       30622
culture     27447
religion    14015
society     13769
economy     11105
Name: theme, dtype: int64

In [41]:
data = pd.concat([data.loc[data.theme == 'world', ['text', 'theme']][:6500], data.loc[data.theme == 'politics', ['text', 'theme']][:9500], 
                 data.loc[data.theme == 'science', ['text', 'theme']][:6000],data.loc[data.theme == 'culture', ['text', 'theme']][:6000], 
                 data.loc[data.theme == 'society', ['text', 'theme']][:8000],data.loc[data.theme == 'economy', ['text', 'theme']][:9000],
                 data.loc[data.theme == 'religion', ['text', 'theme']][:8000]], axis=0)

In [97]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data.text)]

In [100]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [113]:
model.infer_vector([list(data.text)[9]])

array([-0.06359553,  0.068992  ,  0.08132996, -0.00277544, -0.0876869 ],
      dtype=float32)

In [16]:
data.head()

,text,theme
0,"""радикальная партия"" украины предлагает создат...",world
1,президент абхазии рауль хаджимба подписал указ...,world
2,"жизнь одного из двух граждан россии, пострадав...",world
3,неизвестные на одной из виноделен на севере ит...,world
4,глава мид польши витольд ващиковский опроверг ...,world


In [8]:
y_train = data['theme']
X_train = data.drop(['theme'], axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train['vector'], y_train, test_size=0.40, random_state=42)

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
text_clf = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', SGDClassifier(loss='modified_huber', penalty='l2', alpha=1e-3, random_state=40, max_iter=10, tol=None)),])

from sklearn import svm
text_clf = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', svm.SVC(max_iter=7, random_state=40, gamma='scale'))])

In [11]:
from sklearn.linear_model import SGDClassifier
text_clf = SGDClassifier(loss='modified_huber', penalty='l2', alpha=1e-3, random_state=40, max_iter=10, tol=None);

In [12]:
text_clf.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, text_clf.predict(X_test))